<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/CodeBase/llama2_7b_chat_qlora_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 

In [2]:
%%capture
!pip install torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/CSE8803_DLT_Project/

/content/drive/MyDrive/CSE8803_DLT_Project


In [5]:
%ls

ashish_test/  finqa_dataset/          merged_model/  peft-dialogue-summary/
CodeBase/     llama2-docsum-adapter/  Papers/        README.md


In [6]:
dir_path = '/content/drive/MyDrive/finqa_dataset'
import os
os.chdir(dir_path)

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm
from utils import *
import json
import pandas as pd
import torch
# from torch.utils.data import Dataset, DataLoader
import time
import evaluate
from datasets import Dataset, load_dataset
import random

In [8]:
with open(f"{dir_path}/train.json") as input_file:
        train_data = json.load(input_file)

with open(f"{dir_path}/dev.json") as input_file:
        valid_data = json.load(input_file)

with open(f"{dir_path}/test.json") as input_file:
        test_data = json.load(input_file)

In [9]:
assert len(train_data) == 6251, "The train data seems to be off, please check"
assert len(valid_data) == 883, "The validation data seems to be off, please check"
assert len(test_data) == 1147, "The test data seems to be off, please check"

In [13]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.5 MB/s eta 0:00:00


**Creating a joint file for meaningful Table Serialization**

In [10]:
concatenated_df = pd.concat([pd.DataFrame(data)[['pre_text','post_text','filename', 'table_ori','table','id']].assign(dataset=name) for name, data in {'train': train_data, 'valid': valid_data, 'test': test_data}.items()], ignore_index=True)


In [11]:
concatenated_df.shape

(8281, 7)

In [12]:
concatenated_df.to_csv('/content/drive/MyDrive/finqa_dataset/df_combined_table_ser.csv',index=False)

**File written to csv**

***Extracting tables from a pdf report***

In [10]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 47.2 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    # Open the PDF
    doc = fitz.open(pdf_path)

    # Extract text from each page
    text = ""
    for page in doc:
        text += page.get_text()

    doc.close()
    return text

# Replace 'your_pdf_file.pdf' with your PDF file path
pdf_file = 'jpmc_annualreport-2022.pdf'
pdf_text = extract_text_from_pdf(pdf_file)
print(pdf_text)


In [ ]:
import PyPDF2
import pandas as pd

def extract_tables(pdf_path, start_page, end_page):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # Iterate through each page and extract text
        text = ""
        for i in range(start_page - 1, end_page):
            page = reader.pages[i]
            text += page.extract_text()

    # Process the text into a DataFrame
    df = process_text_into_dataframe(text)

    return df

def process_text_into_dataframe(text):
    # Split the text into rows
    rows = text.split('\n')

    # Assuming columns in each row are separated by a specific delimiter
    # Adjust the delimiter based on your PDF's format (e.g., ',', '\t', ';')
    delimiter = ','  # Replace with the actual delimiter
    data = [row.split(delimiter) for row in rows if row.strip()]

    # Convert list of rows into a DataFrame
    df = pd.DataFrame(data)

    return df

# Specify the path to your PDF
pdf_path = 'table.pdf'  # Replace with your PDF file path

# Extract tables
extracted_df = extract_tables(pdf_path, start_page=0, end_page=1)

# Display the DataFrame
print(extracted_df)


In [13]:
train_data[0].keys()

dict_keys(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'table_retrieved', 'text_retrieved', 'table_retrieved_all', 'text_retrieved_all'])

In [14]:
train_data[0]['id']

'ADI/2009/page_49.pdf-1'

In [15]:
train_data[2]['table_ori']

[['Year',
  'Gallons',
  'Average Priceper Gallon',
  'Aircraft FuelExpense',
  'Percent of TotalOperating Expenses'],
 ['2018', '4,447', '$2.23', '$9,896', '23.6%'],
 ['2017', '4,352', '1.73', '7,510', '19.6%'],
 ['2016', '4,347', '1.42', '6,180', '17.6%']]

In [ ]:
train_data[0]

In [14]:
train_data[0]["qa"]

{'question': 'what is the the interest expense in 2009?',
 'answer': '380',
 'explanation': '',
 'ann_table_rows': [],
 'ann_text_rows': [1],
 'steps': [{'op': 'divide1-1', 'arg1': '100', 'arg2': '100', 'res': '1%'},
  {'op': 'divide1-2', 'arg1': '3.8', 'arg2': '#0', 'res': '380'}],
 'program': 'divide(100, 100), divide(3.8, #0)',
 'gold_inds': {'text_1': 'if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million .'},
 'exe_ans': 3.8,
 'tfidftopn': {'text_14': 'dollar , would have on the fair value of our forward exchange contracts as of october 31 , 2009 and november 1 , 2008: .',
  'text_0': 'interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .'},
 'program_re': 'divide(3.8, divide(100, 100))',
 'model_input': [['text_0',
   'interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2

In [10]:
def prepare_data(json_data, verbose=True):
    net=[]
    err_cnt=0
    for example in json_data:
        question = example["qa"]["question"]
        table = example["table"]
        id=example["id"]
        table_ori=example["table_ori"]
        answer_steps=example["qa"]["steps"]
        table_text = ""
        for row in table[1:]:
            this_sent = table_row_to_text(table[0], row)
            table_text += this_sent
        try:
            steps_text = format_steps(example["qa"]["steps"])
            inputs = {"id":id, "context": table_text, "question": question, "answer": steps_text,\
                      "table":table_ori,"answer_steps":answer_steps}
            net.append(inputs)
        except:
            err_cnt+=1
            if verbose:
                print ("-"*25)
                print (example["filename"])
                print (example["qa"]["steps"])
                print ("-"*25+"\n")
    if err_cnt>0:
        print ("Net Errors:",err_cnt)
    return net

In [11]:
## Preparing huggingface dataset
data_splits = {'train': train_data, 'valid': valid_data, 'test': test_data}
datasets = {split: Dataset.from_pandas(pd.DataFrame(prepare_data(data, False))) for split, data in data_splits.items()}

Net Errors: 198
Net Errors: 35
Net Errors: 39


In [17]:
datasets

{'train': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps'],
     num_rows: 6053
 }),
 'valid': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps'],
     num_rows: 848
 }),
 'test': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps'],
     num_rows: 1108
 })}

In [18]:
index=2
datasets['train'][index]

{'id': 'AAL/2018/page_13.pdf-2',
 'context': 'year the 2018 of gallons is 4447 ; the 2018 of average priceper gallon is $ 2.23 ; the 2018 of aircraft fuelexpense is $ 9896 ; the 2018 of percent of totaloperating expenses is 23.6% ( 23.6 % ) ;year the 2017 of gallons is 4352 ; the 2017 of average priceper gallon is 1.73 ; the 2017 of aircraft fuelexpense is 7510 ; the 2017 of percent of totaloperating expenses is 19.6% ( 19.6 % ) ;year the 2016 of gallons is 4347 ; the 2016 of average priceper gallon is 1.42 ; the 2016 of aircraft fuelexpense is 6180 ; the 2016 of percent of totaloperating expenses is 17.6% ( 17.6 % ) ;',
 'question': 'what was the total operating expenses in 2018 in millions',
 'answer': 'Step 1: Divide 9896 by 23.6%. This gives the result: 41932',
 'table': [['Year',
   'Gallons',
   'Average Priceper Gallon',
   'Aircraft FuelExpense',
   'Percent of TotalOperating Expenses'],
  ['2018', '4,447', '$2.23', '$9,896', '23.6%'],
  ['2017', '4,352', '1.73', '7,510', '19.6

**Loading in Llama7B-Chat mode**

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [12]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

# Use the token in your application
# For example, setting an environment variable (optional)
import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token
# Now you can use Hugging Face APIs with the token
# Example: loading a model, accessing API, etc.


Enter your Hugging Face token: ··········


Pushing the Dataset to HuggingFace Hub

In [13]:
index=2
datasets['train'][index]

{'id': 'AAL/2018/page_13.pdf-2',
 'context': 'year the 2018 of gallons is 4447 ; the 2018 of average priceper gallon is $ 2.23 ; the 2018 of aircraft fuelexpense is $ 9896 ; the 2018 of percent of totaloperating expenses is 23.6% ( 23.6 % ) ;year the 2017 of gallons is 4352 ; the 2017 of average priceper gallon is 1.73 ; the 2017 of aircraft fuelexpense is 7510 ; the 2017 of percent of totaloperating expenses is 19.6% ( 19.6 % ) ;year the 2016 of gallons is 4347 ; the 2016 of average priceper gallon is 1.42 ; the 2016 of aircraft fuelexpense is 6180 ; the 2016 of percent of totaloperating expenses is 17.6% ( 17.6 % ) ;',
 'question': 'what was the total operating expenses in 2018 in millions',
 'answer': 'Step 1: Divide 9896 by 23.6%. This gives the result: 41932',
 'table': [['Year',
   'Gallons',
   'Average Priceper Gallon',
   'Aircraft FuelExpense',
   'Percent of TotalOperating Expenses'],
  ['2018', '4,447', '$2.23', '$9,896', '23.6%'],
  ['2017', '4,352', '1.73', '7,510', '19.6

In [21]:
# model_id =  "NousResearch/Llama-2-7b-hf"
model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",use_auth_token=hf_token)

tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [17]:
index=2
datasets['train'][index]

{'id': 'AAL/2018/page_13.pdf-2',
 'context': 'year the 2018 of gallons is 4447 ; the 2018 of average priceper gallon is $ 2.23 ; the 2018 of aircraft fuelexpense is $ 9896 ; the 2018 of percent of totaloperating expenses is 23.6% ( 23.6 % ) ;year the 2017 of gallons is 4352 ; the 2017 of average priceper gallon is 1.73 ; the 2017 of aircraft fuelexpense is 7510 ; the 2017 of percent of totaloperating expenses is 19.6% ( 19.6 % ) ;year the 2016 of gallons is 4347 ; the 2016 of average priceper gallon is 1.42 ; the 2016 of aircraft fuelexpense is 6180 ; the 2016 of percent of totaloperating expenses is 17.6% ( 17.6 % ) ;',
 'question': 'what was the total operating expenses in 2018 in millions',
 'answer': 'Step 1: Divide 9896 by 23.6%. This gives the result: 41932'}

In [22]:
def format_instruction(context: str, question: str, answer: str):
	return f"""### Instruction:
Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
{context.strip()}

### Question:
{question.strip()}

### Answer:
{answer}
""".strip()

In [23]:
def generate_instruction_dataset(data_point):

    return {
        "context": data_point["context"],
        "question": data_point["question"],
        "answer": data_point["answer"],
        "table": data_point["table"],
        "input_prompt": format_instruction(data_point["context"],data_point["question"],data_point["answer"])
    }

In [24]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset)
    )

In [25]:
## APPLYING PREPROCESSING ON WHOLE DATASET
datasets["train"] = process_dataset(datasets["train"])
datasets["test"] = process_dataset(datasets["test"])
datasets["valid"] = process_dataset(datasets["valid"])

Map:   0%|          | 0/6053 [00:00<?, ? examples/s]

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

Map:   0%|          | 0/848 [00:00<?, ? examples/s]

In [22]:
index=0
print(datasets["train"]["input_prompt"][index])

### Instruction:
Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
reporting unit the retail brokerage of december 31 2012 goodwill is $ 1791.8 ; the retail brokerage of december 31 2012 % ( % ) of fair value to book value is 190% ( 190 % ) ;reporting unit the market making of december 31 2012 goodwill is 142.4 ; the market making of december 31 2012 % ( % ) of fair value to book value is 115% ( 115 % ) ;reporting unit the total goodwill of december 31 2012 goodwill is $ 1934.2 ; the total goodwill of december 31 2012 % ( % ) of fair value to book value is ;

### Question:
what percentage of total goodwill is comprised of market making at december 31 2012?

### Answer:
Step 1: Divide 142.4 by 1934.2. This gives the result: 7%


In [23]:
question1= "What is the ratio of Net Income in 2022 to net income in 2021?"
question2="What is the Net Income of 2022?"
question3="What is the increase in net income from 2020 to 2022?"
question4="What is the gross net income over 2020-2022?"

context= ''' 'the Net income of 2022 is 37,676 ; '
 'the Net income of is $ ; the Net '
 'income of 2021 is 48,334 ; the Net '
 'income of is $ ; the Net income of '
 '2020 is 29,131 ;(in millions, except '
 'per share, ratio data and headcount) '
 'the Per common share data of is nan '
 '; the Per common share data of 2022 '
 'is nan ; the Per common share data '
 'of is nan ; the Per common share '
 'data of 2021 is nan ; the Per common '
 'share data of is nan ; the Per '
 'common share data of 2020 is nan' '''

answer=''

In [28]:
train_data[0]['table_ori']

[['', 'October 31, 2009', 'November 1, 2008'],
 ['Fair value of forward exchange contracts asset (liability)',
  '$6,427',
  '$(23,158)'],
 ['Fair value of forward exchange contracts after a 10% unfavorable movement in foreign currency exchange rates asset (liability)',
  '$20,132',
  '$(9,457)'],
 ['Fair value of forward exchange contracts after a 10% favorable movement in foreign currency exchange rates liability',
  '$(6,781)',
  '$(38,294)']]

In [31]:
datasets['train'][0]['answer'],datasets['train'][1]['answer'],datasets['train'][2]['answer'],datasets['train'][4]['answer']

('Step 1: Divide 142.4 by 1934.2. This gives the result: 7%',
 'Step 1: Divide 208 by 2270. This gives the result: 9%',
 'Step 1: Divide 1557 by 35%. This gives the result: 4448.6',
 'Step 1: Subtract 18.9 from 29.4. This gives the result: 10.5 ####### Step 2: Divide 10.5 by 18.9. This gives the result: 56%')

In [39]:
import warnings
warnings.filterwarnings('ignore')
# index = 10 (question not understood clearly), 11 (answer and context irrelevant to the question being asked)
index = [0,2,4,1]

context_table=datasets['train'][index[0]]['table']
context = datasets['train'][index[0]]['context'],datasets['train'][index[1]]['context'],datasets['train'][index[2]]['context'],datasets['train'][index[3]]['context']
answer = datasets['train'][index[0]]['answer'],datasets['train'][index[1]]['answer'],datasets['train'][index[2]]['answer'],datasets['train'][index[3]]['answer']
question = datasets['train'][index[0]]['question'],datasets['train'][index[1]]['question'],datasets['train'][index[2]]['question'],datasets['train'][index[3]]['question']
dash_line = '-'.join('' for x in range(100))
prompt=''
for i,j in enumerate((index[:3])):

  prompt+=f"""
Answer the question using the information from the context below. Think step by step and solve the problems in concise steps.

### Context:
{context[i]}

### Question:
{question[i]}

### Answer:
json_answer:{"steps":{answer[i]}}
"""
  prompt+=(dash_line)

prompt+= f"""
Answer the question using the information from the context below. Think step by step and solve the problems in concise steps.

### Context:
{context[3]}

### Question:
{question[3]}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=128,
    )[0],
    skip_special_tokens=True
)


print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer[3]}\n')
# print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the question using the information from the context below. Think step by step and solve the problems in concise steps. 

### Context:
reporting unit the retail brokerage of december 31 2012 goodwill is $ 1791.8 ; the retail brokerage of december 31 2012 % ( % ) of fair value to book value is 190% ( 190 % ) ;reporting unit the market making of december 31 2012 goodwill is 142.4 ; the market making of december 31 2012 % ( % ) of fair value to book value is 115% ( 115 % ) ;reporting unit the total goodwill of december 31 2012 goodwill is $ 1934.2 ; the total goodwill of december 31 2012 % ( % ) of fair value to book value is ;

### Question:
what percentage of total goodwill is comprised of market making at december 31 2012?

### Answer:
Step 1: Divide 142.4 by 1934.2. This gives the result: 7%
---------------------------------------------------------------------------

In [47]:
import warnings
warnings.filterwarnings('ignore')
# index = 10 (question not understood clearly), 11 (answer and context irrelevant to the question being asked)
index = 0

context_table=datasets['train'][index]['table']
context = datasets['train'][index]['context']
answer = datasets['train'][index]['answer']
question = datasets['train'][index]['question']

prompt = f"""
Answer the question using the information from the context below. Think step by step and answer the question briefly.
Return the variables and arithmetic operation used in the last step in a format that is easily parseable for calculating.
### Context:
{context}

### Question:
{question}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=512,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the question using the information from the context below. Think step by step and answer the question briefly. 
Return the variables and arithmetic operation used in the last step in a format that is easily parseable for calculating.
### Context:
reporting unit the retail brokerage of december 31 2012 goodwill is $ 1791.8 ; the retail brokerage of december 31 2012 % ( % ) of fair value to book value is 190% ( 190 % ) ;reporting unit the market making of december 31 2012 goodwill is 142.4 ; the market making of december 31 2012 % ( % ) of fair value to book value is 115% ( 115 % ) ;reporting unit the total goodwill of december 31 2012 goodwill is $ 1934.2 ; the total goodwill of december 31 2012 % ( % ) of fair value to book value is ;

### Question:
what percentage of total goodwill is comprised of market making at december 31 2012?

### Answer:

-------------------

In [53]:
import warnings
warnings.filterwarnings('ignore')
# index = 10 (question not understood clearly), 11 (answer and context irrelevant to the question being asked)
index = 4

context_table=datasets['train'][index]['table']
context = datasets['train'][index]['context']
answer = datasets['train'][index]['answer']
question = datasets['train'][index]['question']

prompt = f"""
Answer the question using the information from the context below. Think step by step and answer the question briefly.
### Context:
{context}

### Question:
{question}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=256,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the question using the information from the context below. Think step by step and answer the question briefly. 
### Context:
the expected volatility of 2009 is 57.1% ( 57.1 % ) ; the expected volatility of 2008 is 56.4% ( 56.4 % ) ; the expected volatility of 2007 is 28.3% ( 28.3 % ) ;the risk-free interest rate of 2009 is 1.9% ( 1.9 % ) ; the risk-free interest rate of 2008 is 2.4% ( 2.4 % ) ; the risk-free interest rate of 2007 is 4.5% ( 4.5 % ) ;the dividend yield of 2009 is 0.0% ( 0.0 % ) ; the dividend yield of 2008 is 2.7% ( 2.7 % ) ; the dividend yield of 2007 is 1.1% ( 1.1 % ) ;the expected life ( years ) of 2009 is 3.9 ; the expected life ( years ) of 2008 is 5.5 ; the expected life ( years ) of 2007 is 6.5 ;

### Question:
what is the percent change in number of shares purchased by employees between 2008 and 2009?

### Answer:

----------------------------

In [50]:
answer

'Step 1: Divide 208 by 2270. This gives the result: 9%'

In [51]:
context = 'There are 2 apples and 3 oranges in a basket. I bought a dozen of apples today.'
answer = ''
question = 'What is the total number of fruits that I have?'

prompt = f"""
Answer the question using the information from the context below. Think step by step and generate the final answer.

### Context:
{context}

### Question:
{question}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=1024,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the question using the information from the context below. Think step by step and generate the final answer.

### Context:
There are 2 apples and 3 oranges in a basket. I bought a dozen of apples today.

### Question:
What is the total number of fruits that I have?

### Answer:

---------------------------------------------------------------------------------------------------
BASELINE ANSWER:


---------------------------------------------------------------------------------------------------
MODEL GENERATED ANSWER - ZERO SHOT:

Answer the question using the information from the context below. Think step by step and generate the final answer.

### Context:
There are 2 apples and 3 oranges in a basket. I bought a dozen of apples today.

### Question:
What is the total number of fruits that I have?

### Answer:
The total number of fruits that you have is:

Apples: 2 

In [52]:
table = ['''[['In millions', 'InterestOnlyProduct', 'PrincipalandInterestProduct'],
 ['2013', '$1,338', '$221'],
 ['2014', '2,048', '475'],
 ['2015', '2,024', '654'],
 ['2016', '1,571', '504'],
 ['2017', '3,075', '697'],
 ['2018 and thereafter', '5,497', '4,825'],
 ['Total (a)', '$15,553', '$7,376']]''',
    '''[['Year',
  'Gallons',
  'Average Priceper Gallon',
  'Aircraft FuelExpense',
  'Percent of TotalOperating Expenses'],
 ['2018', '4,447', '$2.23', '$9,896', '23.6%'],
 ['2017', '4,352', '1.73', '7,510', '19.6%'],
 ['2016', '4,347', '1.42', '6,180', '17.6%']]
''','''[['(In Millions)', 'Dec 28,2013', 'Dec 29,2012'],
 ['Available-for-sale investments', '$18,086', '$14,001'],
 ['Cash', '854', '593'],
 ['Equity method investments', '1,038', '992'],
 ['Loans receivable', '1,072', '979'],
 ['Non-marketable cost method investments', '1,270', '1,202'],
 ['Reverse repurchase agreements', '800', '2,850'],
 ['Trading assets', '8,441', '5,685'],
 ['Total cash and investments', '$31,561', '$26,302']]''']
instruction = ['''Given a table as a 2D list above, understand the semantics and structure through the values present. Produce a serialized text that lists all the information present in the table.''']*2 + ['''Follow the pattern above through the examples given and produce a serialized text that lists all the information present in the table in a concise manner.''']

answers=['''As of Dec 28, 2013, the available-for-sale investments were valued at $18,086 million, cash reserves were at $854 million, equity method investments stood at $1,038 million, loans receivable were at $1,072 million, non-marketable cost method investments were valued at $1,270 million, reverse repurchase agreements totaled $800 million, and trading assets were at $8,441 million. This resulted in a total cash and investments figure of $31,561 million. In comparison, on Dec 29, 2012, available-for-sale investments were $14,001 million, cash at $593 million, equity method investments at $992 million, loans receivable at $979 million, non-marketable cost method investments at $1,202 million, reverse repurchase agreements at $2,850 million, and trading assets at $5,685 million, with a total cash and investments of $26,302 million.''',\
         '''In 2018, 4,447 gallons of fuel were used at an average price per gallon of $2.23, resulting in an aircraft fuel expense of $9,896, which constituted 23.6% of the total operating expenses. In 2017, the fuel usage was 4,352 gallons with an average price of $1.73 per gallon, leading to an aircraft fuel expense of $7,510 and accounting for 19.6% of the total operating expenses. In 2016, a total of 4,347 gallons were used, the average price per gallon was $1.42, the aircraft fuel expense amounted to $6,180, and it represented 17.6% of the total operating expenses.''']

prompt = f"""
### Table:
{table[1]}

### Instruction:
{instruction[0]}

### Answer:
"""
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"].to('cuda'),
        temperature=0.2,
        do_sample=True,
        max_new_tokens=1024,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
# print(dash_line)
# print(f'INPUT PROMPT:\n{prompt}')
# print(dash_line)
# print(f'BASELINE ANSWER:\n{answer}\n')
# print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

MODEL GENERATED ANSWER - ZERO SHOT:

### Table:
[['Year',
  'Gallons',
  'Average Priceper Gallon',
  'Aircraft FuelExpense',
  'Percent of TotalOperating Expenses'],
 ['2018', '4,447', '$2.23', '$9,896', '23.6%'],
 ['2017', '4,352', '1.73', '7,510', '19.6%'],
 ['2016', '4,347', '1.42', '6,180', '17.6%']]


### Instruction:
Given a table as a 2D list above, understand the semantics and structure through the values present. Produce a serialized text that lists all the information present in the table.

### Answer:
The serialized text of the table is:

"Year Gallons Average Priceper Gallon Aircraft FuelExpense Percent of TotalOperating Expenses
2018 4447 $2.23 $9896 23.6%
2017 4352 1.73 7510 19.6%
2016 4347 1.42 6180 17.6%"

Note: The table is represented as a list of lists, where each inner list contains the values for a particular column of the table.


In [ ]:
table='''[['In millions', 'InterestOnlyProduct', 'PrincipalandInterestProduct'],
 ['2013', '$1,338', '$221'],
 ['2014', '2,048', '475'],
 ['2015', '2,024', '654'],
 ['2016', '1,571', '504'],
 ['2017', '3,075', '697'],
 ['2018 and thereafter', '5,497', '4,825'],
 ['Total (a)', '$15,553', '$7,376']]'''

instruction = '''Given a table as a 2D list above, understand the semantics and structure through the values present. Produce a serialized text that lists all the information present in the table in a concise manner.'''
prompt = f"""
### Table:{table}

### Instruction:
{instruction}

### Answer:
"""